# RNN 계열 실습


1) 실습 목적 <br>
이번 실습에서는 연속적인 데이터를 처리하기 위한 구조로, Transformer 등장 이전에 가장 활발히 사용된 RNN (Recurrent Neural Network)과 RNN의 변형인  LSTM (Long Short-Term Memory), GRU (Gated Recurrent Unit)에 대해 pytorch를 사용하여 구현하고 이해해봅니다<br>
더 나아가, 연속적인 데이터를 활용하여 이를 학습해보고, RNN에서 발생하는 Gradient Vanishing / Exploding 문제를 직접 확인해보고 LSTM과 GRU가 이를 극복할 수 있는지 직접 확인해봅니다 <br>

</font></b>

 2) 수강 목표
  * RNN, LSTM, GRU를 구현하고 작동 방식을 이해한다
  * One-to-one / One-to-many / Many-to-one / Many-to-many RNN을 구현하고 각 구조를 이해한다
  * 각 과정에서 일어나는 연산과 input/output 형태에 대해 이해한다
  * RNN에서 발생하는 Gradient Vanishing / Exploding 현상을 직접 관찰하고, LSTM과 GRU가 극복 가능한지 확인한다
  * RNN, LSTM, GRU를 이해하고 사용할 수 있다


### 실습 목차
* 1. RNN 실습
  * 1-1. RNN 구현
  * 1-2. RNN 학습
* 2. LSTM과 GRU 실습
  * 2-1. LSTM과 GRU 구현
  * 2-2. LSTM과 GRU 학습
* 3. One-to-one / One-to-many / Many-to-one / Many-to-many RNNs 구현 및 실습
  * 3-1. One-to-one RNN 실습
  * 3-2. One-to-Many RNN 실습
  * 3-3. Many-to-One RNN 실습
  * 3-4. Many-to-Many RNN 실습
* 4. Gradient Vanishing / Exploding

**※ 코드를 실행 전에 상단에 `런타임` →  `런타임 유형 변경` →  `하드웨어 가속기`를 `GPU`로 설정해주세요!**

In [ ]:
# 라이브러리 import
import torch
import torch.nn as nn
import numpy as np

# GPU 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 1. RNN 실습

```
💡 목차 개요 : pytorch를 통해 RNN을 구현 및 학습해보고, RNN의 동작 원리를 이해합니다
```

- 1-1. RNN 구현
- 1-2. RNN 학습


#### RNN 구현

- 이 코드는 간단한 RNN 모델을 정의합니다.
  - `__init__` 함수에서는 필요한 레이어를 정의하고
  - `forward` 함수에서는 이 레이어들을 어떻게 연결할지를 정의합니다

In [ ]:
class SimpleRNN(nn.Module): # SimpleRNN 클래스 선언
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(SimpleRNN, self).__init__() # nn.Module의 초기화 함수 상속
        self.M = n_hidden # 은닉 상태(hidden state)의 크기를 지정
        self.D = n_inputs # 입력 차원의 크기 지정
        self.K = n_outputs # 출력 차원의 크기 지정
        self.rnn = nn.RNN( # RNN 모듈을 생성
            input_size=self.D, # 입력 차원의 크기 지정
            hidden_size=self.M, # 은닉 상태의 크기 지정
            nonlinearity='tanh', # 활성화 함수로 tanh를 사용
            batch_first=True) # 배치 차원이 먼저 오도록 설정
        self.fc = nn.Linear(self.M, self.K) # 출력을 위한 선형 변환을 정의

    def forward(self, X): # 순전파 함수를 정의
        # initial hidden states
        h0 = torch.zeros(1, X.size(0), self.M).to(X.device) # 초기 은닉 상태를 0으로 설정

        # get RNN unit output
        out, _ = self.rnn(X, h0) # RNN에 입력을 전달하고 출력을 받음

        # we only want h(T) at the final time step
        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

#### RNN 학습

RNN을 학습시키기 위해서는 먼저 데이터를 준비해야 합니다. 여기서는 간단한 예제를 위해 임의의 데이터를 사용하겠습니다.

이 코드는 RNN 모델을 학습시키는 과정을 보여줍니다. 각 에포크 (epoch)에서는 입력 데이터의 시퀀스를 순회하면서 모델의 출력과 실제 출력을 비교하여 손실 (loss)을 계산합니다. 그런 다음 이 손실을 이용하여 모델의 가중치를 업데이트합니다.


In [ ]:
model = SimpleRNN(n_inputs=2, n_hidden=20, n_outputs=2).to(device) # SimpleRNN 모델을 생성하고, GPU로 전송
criterion = nn.CrossEntropyLoss() # 손실 함수로 CrossEntropyLoss를 사용
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘으로 Adam을 사용

# 더미 입력 데이터 예제
inputs = torch.from_numpy(np.array([[[1, 2], [3, 4], [5, 6]]], dtype=np.float32)).to(device)

for epoch in range(300): # 300번의 에폭 동안 학습을 진행
    model.zero_grad() # 기울기를 0으로 초기화
    outputs = model(inputs) # 모델에 입력을 전달하고 출력을 받음
    loss = criterion(outputs, torch.tensor([1]).to(device))  # 더미 타겟 데이터로 손실(loss)을 계산
    loss.backward() # 역전파를 통해 기울기를 계산
    optimizer.step() # 최적화 알고리즘을 통해 파라미터를 업데이트

    if (epoch+1) % 30 == 0: # 30 에폭마다 손실을 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 300, loss.item()))

Epoch [30/300], Loss: 0.1273
Epoch [60/300], Loss: 0.0485
Epoch [90/300], Loss: 0.0270
Epoch [120/300], Loss: 0.0179
Epoch [150/300], Loss: 0.0130
Epoch [180/300], Loss: 0.0100
Epoch [210/300], Loss: 0.0081
Epoch [240/300], Loss: 0.0067
Epoch [270/300], Loss: 0.0057
Epoch [300/300], Loss: 0.0049



## 2. LSTM과 GRU 실습

```
💡 목차 개요 : pytorch를 통해 RNN의 변형인 LSTM과 GRU를 구현 및 학습해보고, 동작 원리를 이해합니다
```
- 2-1. LSTM과 GRU 구현
- 2-2. LSTM과 GRU 학습


##### LSTM과 GRU 구현


LSTM(Long Short-Term Memory)과 GRU(Gated Recurrent Unit)는 RNN의 변형으로, gradient vanishing 또는 exploding 문제를 완화하는 메커니즘이 추가되었습니다

LSTM은 `torch.nn.LSTM`을 사용하여 구현할 수 있습니다

`torch.nn.LSTM`의 초기화 인자는 다음과 같습니다:


- `input_size` : LSTM 모듈의 입력 차원의 크기를 지정합니다. 이는 각 입력 요소 벡터의 크기를 의미합니다. 여기서 self.D는 입력 차원의 크기를 나타내는 변수입니다

- `hidden_size` : LSTM 모듈의 은닉 상태의 크기를 지정합니다. 이는 LSTM의 은닉층의 뉴런 수를 의미합니다. 여기서 self.M는 은닉 상태의 크기를 나타내는 변수입니다

- `num_layers` : LSTM 셀을 몇 층으로 쌓을 것인지를 결정합니다. 기본값은 1입니다

- `batch_first` : 입력 데이터의 형태를 결정하는 옵션입니다. True로 설정하면 입력 데이터의 형태가 (배치 크기, 시퀀스 길이, 입력 차원)이 됩니다. False로 설정하면 (시퀀스 길이, 배치 크기, 입력 차원)이 됩니다


In [ ]:
class LSTM(nn.Module): # LSTM 클래스 선언
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(LSTM, self).__init__() # nn.Module의 초기화 함수 상속
        self.D = n_inputs # 입력 차원의 크기 지정
        self.M = n_hidden # 은닉 상태(hidden state)의 크기를 지정
        self.K = n_outputs # 출력 차원의 크기 지정
        self.lstm = nn.LSTM( # LSTM 모듈을 생성
            input_size=self.D, # 입력 차원의 크기 지정
            hidden_size=self.M, # 은닉 상태의 크기 지정
            batch_first=True) # 배치 차원이 먼저 오도록 설정
        self.fc = nn.Linear(self.M, self.K) # 출력을 위한 선형 변환을 정의

    def forward(self, X): # 순전파 함수를 정의
        h0 = torch.zeros(1, X.size(0), self.M).to(X.device) # 초기 은닉 상태를 0으로 설정
        c0 = torch.zeros(1, X.size(0), self.M).to(X.device) # LSTM의 초기 cell state를 0으로 설정

        # get RNN unit output
        out, _ = self.lstm(X, (h0, c0)) # LSTM에 입력을 전달하고 출력을 받음

        # we only want h(T) at the final time step
        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

GRU는 `torch.nn.GRU`을 사용하여 구현할 수 있습니다

`torch.nn.GRU`의 초기화 인자는 다음과 같습니다:

- `input_size` : GRU 모듈의 입력 차원의 크기를 지정합니다. 이는 각 입력 요소 벡터의 크기를 의미합니다

- `hidden_size` : GRU 모듈의 은닉 상태의 크기를 지정합니다. 이는 GRU의 은닉층의 뉴런 수를 의미합니다

- `num_layers` : GRU 셀을 몇 층으로 쌓을 것인지를 결정합니다. 기본값은 1입니다

- `batch_first` : 입력 데이터의 형태를 결정하는 옵션입니다. True로 설정하면 입력 데이터의 형태가 (배치 크기, 시퀀스 길이, 입력 차원)이 됩니다. False로 설정하면 (시퀀스 길이, 배치 크기, 입력 차원)이 됩니다



In [ ]:
class GRU(nn.Module): # GRU 클래스 선언
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(GRU, self).__init__() # nn.Module의 초기화 함수 상속
        self.D = n_inputs # 입력 차원의 크기 지정
        self.M = n_hidden # 은닉 상태(hidden state)의 크기를 지정
        self.K = n_outputs # 출력 차원의 크기 지정
        self.gru = nn.GRU( # GRU 모듈을 생성
            input_size=self.D, # 입력 차원의 크기 지정
            hidden_size=self.M, # 은닉 상태의 크기 지정
            batch_first=True) # 배치 차원이 먼저 오도록 설정
        self.fc = nn.Linear(self.M, self.K) # 출력을 위한 선형 변환을 정의

    def forward(self, X): # 순전파 함수를 정의
        # initial hidden states
        h0 = torch.zeros(1, X.size(0), self.M).to(X.device) # 초기 은닉 상태를 0으로 설정

        # get RNN unit output
        out, _ = self.gru(X, h0) # GRU에 입력을 전달하고 출력을 받음

        # we only want h(T) at the final time step
        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

#### LSTM과 GRU 학습

In [ ]:
# LSTM 학습
model = LSTM(n_inputs=2, n_hidden=20, n_outputs=2).to(device) # LSTM 모델을 생성
criterion = nn.CrossEntropyLoss() # 손실 함수로 CrossEntropyLoss를 사용
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘으로 Adam을 사용

for epoch in range(300): # 300회의 에포크동안 학습을 진행
    model.zero_grad() # 기울기를 0으로 초기화
    outputs = model(inputs) # 모델에 입력을 전달하고 출력을 받음
    loss = criterion(outputs, torch.tensor([1]).to(device))  # 예시로 사용할 목표 텐서 생성
    loss.backward() # 역전파를 수행하여 기울기를 계산
    optimizer.step() # 최적화 알고리즘을 통해 파라미터 업데이트

    if (epoch+1) % 30 == 0: # 30 에포크마다 손실을 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 300, loss.item()))

Epoch [30/300], Loss: 0.2008
Epoch [60/300], Loss: 0.0616
Epoch [90/300], Loss: 0.0230
Epoch [120/300], Loss: 0.0117
Epoch [150/300], Loss: 0.0073
Epoch [180/300], Loss: 0.0052
Epoch [210/300], Loss: 0.0039
Epoch [240/300], Loss: 0.0031
Epoch [270/300], Loss: 0.0026
Epoch [300/300], Loss: 0.0022


In [ ]:
# GRU 학습
model = GRU(n_inputs=2, n_hidden=20, n_outputs=2).to(device) # GRU 모델 인스턴스 생성
criterion = nn.CrossEntropyLoss() # 손실 함수로 CrossEntropyLoss를 사용
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘으로 Adam을 사용

for epoch in range(300):
    model.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs, torch.tensor([1]).to(device))  # A dummy target example
    loss.backward()
    optimizer.step()

    if (epoch+1) % 30 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 300, loss.item()))

Epoch [30/300], Loss: 0.3474
Epoch [60/300], Loss: 0.1188
Epoch [90/300], Loss: 0.0548
Epoch [120/300], Loss: 0.0333
Epoch [150/300], Loss: 0.0233
Epoch [180/300], Loss: 0.0176
Epoch [210/300], Loss: 0.0139
Epoch [240/300], Loss: 0.0113
Epoch [270/300], Loss: 0.0094
Epoch [300/300], Loss: 0.0079



## 3. One-to-one / One-to-many / Many-to-one / Many-to-many RNNs 구현 및 실습

```
💡 목차 개요 : pytorch를 통해 One-to-one / One-to-many / Many-to-one / Many-to-many RNNs을 구현하고 학습하여 동작 원리를 이해합니다
```
- 3-1. One-to-one RNN 실습
- 3-2. One-to-Many RNN 실습
- 3-3. Many-to-One RNN 실습
- 3-4. Many-to-Many RNN 실습

#### One-to-one RNN 실습
One-to-one RNN을 구현해보고 학습해봅시다

<img src="https://github.com/js-lee-AI/assets/assets/60927808/6edb2a60-009f-4219-aaea-02df147389b1" height="300">


# One-to-One
이 모델은 각 입력에 대해 하나의 출력을 생성하는 가장 기본적인 RNN 구조입니다.
예를 들어, 주어진 숫자의 제곱을 예측하는 문제를 해결할 수 있습니다.
각 입력 숫자에 대해 하나의 출력(입력 숫자의 제곱)을 생성합니다.
One-to-one 구조의 RNN 모델은 주식 가격 예측 등에 사용될 수 있습니다

In [ ]:
# One-to-One
X = np.random.randint(1, 5, size=(1000, 1, 1)) # 입력 데이터 생성, 1~4 사이의 정수 1000개를 랜덤하게 생성
Y = np.square(X) # 타겟 데이터 생성, 입력 데이터의 제곱을 타겟으로 설정

X = torch.from_numpy(X.astype(np.float32)).to(device) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)).squeeze(-1).to(device) # 타겟 데이터를 텐서로 변환

model = SimpleRNN(n_inputs=1, n_hidden=40, n_outputs=1).to(device) # 모델 생성
criterion = nn.MSELoss() # 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘 설정

for epoch in range(4000): # 4000회 반복하여 학습
    model.zero_grad() # 기울기 초기화
    outputs = model(X) # 모델에 입력 데이터 전달하여 예측값 계산
    loss = criterion(outputs, Y) # 예측값과 타겟값을 이용하여 손실 계산
    loss.backward() # 역전파 수행
    optimizer.step() # 가중치 업데이트

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device) # 테스트 데이터 생성
print(f"Input: 2.0, Output: {model(X_test).item()}, 정답: {np.square(2.0)}") # 테스트 데이터에 대한 예측값 출력

Epoch [100/4000], Loss: 50.7856
Epoch [200/4000], Loss: 25.2362
Epoch [300/4000], Loss: 19.1932
Epoch [400/4000], Loss: 14.2959
Epoch [500/4000], Loss: 9.1926
Epoch [600/4000], Loss: 5.3577
Epoch [700/4000], Loss: 3.4169
Epoch [800/4000], Loss: 2.7019
Epoch [900/4000], Loss: 2.4273
Epoch [1000/4000], Loss: 2.2608
Epoch [1100/4000], Loss: 2.1212
Epoch [1200/4000], Loss: 1.9904
Epoch [1300/4000], Loss: 1.8619
Epoch [1400/4000], Loss: 1.7322
Epoch [1500/4000], Loss: 1.5993
Epoch [1600/4000], Loss: 1.4623
Epoch [1700/4000], Loss: 1.3213
Epoch [1800/4000], Loss: 1.1776
Epoch [1900/4000], Loss: 1.0332
Epoch [2000/4000], Loss: 0.8910
Epoch [2100/4000], Loss: 0.7542
Epoch [2200/4000], Loss: 0.6258
Epoch [2300/4000], Loss: 0.5086
Epoch [2400/4000], Loss: 0.4046
Epoch [2500/4000], Loss: 0.3148
Epoch [2600/4000], Loss: 0.2395
Epoch [2700/4000], Loss: 0.1781
Epoch [2800/4000], Loss: 0.1294
Epoch [2900/4000], Loss: 0.0918
Epoch [3000/4000], Loss: 0.0636
Epoch [3100/4000], Loss: 0.0430
Epoch [3200/4

#### One-to-Many RNN 실습
One-to-Many RNN을 구현해보고 학습해봅시다

<img src="https://github.com/js-lee-AI/assets/assets/60927808/8dac773c-b0f8-417d-b3f2-7705c9e14e51" width="500" height="300">

# One-to-Many
One-to-Many 구조의 모델은 하나의 입력에 대해 여러 개의 출력을 생성하는 구조입니다.
예를 들어, 주어진 숫자의 배수를 예측하는 문제를 해결할 수 있습니다.
하나의 입력 숫자에 대해 여러 개의 출력(입력 숫자의 배수)을 생성합니다.
이러한 모델은 이미지 캡셔닝(하나의 이미지에 대한 여러 단어의 설명 생성), 음성 합성(하나의 텍스트 입력에 대한 여러 개의 음성 출력 생성) 등에 사용됩니다

In [ ]:
#### 입력의 배수 10개를 타겟으로 설정
# 아래의 One-to-Many는 하나의 숫자 입력 데이터를 받고, 입력된 데이터의 배수 10개를 예측하는 RNN 모델을 구현합니다.

# One-to-Many
class SimpleRNNOne2Many(nn.Module): # One-to-Many 모델 클래스 선언
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(SimpleRNNOne2Many, self).__init__()
        self.D = n_inputs
        self.M = n_hidden
        self.K = n_outputs
        self.rnn = nn.RNN(
            input_size=self.D,
            hidden_size=self.M,
            nonlinearity='tanh',
            batch_first=True) # RNN 모듈 생성
        self.fc = nn.Linear(self.M, self.K) # 선형 변환 정의

    def forward(self, X): # 순전파 함수 정의
        h0 = torch.zeros(1, X.size(0), self.M).to(X.device) # 초기 은닉 상태를 0으로 설정
        out, _ = self.rnn(X, h0) # RNN에 입력을 전달하고 출력을 받음
        out = self.fc(out) # 출력에 선형 변환을 수행
        return out.view(-1, 10) # 출력을 적절한 형태로 변환

X = np.random.randint(1, 5, size=(1000, 1, 1)) # 입력 데이터 생성, 1~4 사이의 정수 1000개를 랜덤하게 생성
Y = np.array([[i*j for i in range(1, 11)] for j in X.squeeze()]) # 타겟 데이터 생성, 입력의 배수 10개를 타겟으로 설정

X = torch.from_numpy(X.astype(np.float32)).to(device) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)).to(device) # 타겟 데이터를 텐서로 변환

model = SimpleRNNOne2Many(n_inputs=1, n_hidden=40, n_outputs=10).to(device) # 모델 생성
criterion = nn.MSELoss() # 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘 설정

for epoch in range(4000): # 4000회 반복하여 학습
    model.zero_grad() # 기울기 초기화
    outputs = model(X) # 모델에 입력 데이터 전달하여 예측값 계산
    loss = criterion(outputs.squeeze(), Y) # 예측값과 타겟값을 이용하여 손실 계산
    loss.backward() # 역전파 수행
    optimizer.step() # 가중치 업데이트

    if (epoch+1) % 100 == 0: # 100회마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[2.0]]], dtype=torch.float32).to(device) # 테스트 데이터 생성
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: 2.0")
output = [round(num, 1) for num in model(X_test).squeeze().tolist()] # 테스트 데이터에 대한 예측값 계산
answer = list(range(2, 21, 2)) # 정답 리스트 생성

for o, a in zip(output, answer): # 예측값과 정답을 비교하여 출력
    print(f"Output: {o}, 정답: {a}")

Epoch [100/4000], Loss: 208.2682
Epoch [200/4000], Loss: 115.4876
Epoch [300/4000], Loss: 69.8474
Epoch [400/4000], Loss: 49.2465
Epoch [500/4000], Loss: 38.6924
Epoch [600/4000], Loss: 31.7193
Epoch [700/4000], Loss: 25.4721
Epoch [800/4000], Loss: 19.3838
Epoch [900/4000], Loss: 14.0058
Epoch [1000/4000], Loss: 9.8412
Epoch [1100/4000], Loss: 6.9928
Epoch [1200/4000], Loss: 5.2137
Epoch [1300/4000], Loss: 4.1409
Epoch [1400/4000], Loss: 3.4691
Epoch [1500/4000], Loss: 3.0042
Epoch [1600/4000], Loss: 2.6453
Epoch [1700/4000], Loss: 2.3467
Epoch [1800/4000], Loss: 2.0875
Epoch [1900/4000], Loss: 1.8563
Epoch [2000/4000], Loss: 1.6464
Epoch [2100/4000], Loss: 1.4538
Epoch [2200/4000], Loss: 1.2761
Epoch [2300/4000], Loss: 1.1120
Epoch [2400/4000], Loss: 0.9611
Epoch [2500/4000], Loss: 0.8231
Epoch [2600/4000], Loss: 0.6981
Epoch [2700/4000], Loss: 0.5862
Epoch [2800/4000], Loss: 0.4874
Epoch [2900/4000], Loss: 0.4014
Epoch [3000/4000], Loss: 0.3276
Epoch [3100/4000], Loss: 0.2653
Epoch 

#### Many-to-One RNN 실습
Many-to-one RNN을 구현해보고 학습해봅시다

<img src="https://github.com/js-lee-AI/assets/assets/60927808/0b844a30-397a-4af2-b06d-163a7aa06a14" width="400">

# Many-to-One
Many-to-One 구조의 모델은 여러 개의 입력에 대해 하나의 출력을 생성합니다.
예를 들어, 주어진 숫자 리스트의 합을 예측하는 문제를 해결할 수 있습니다.
여러 개의 입력 숫자에 대해 하나의 출력(입력 숫자의 합)을 생성합니다.
Many-to-One 구조의 모델은 감성 분석(여러 단어로 이루어진 텍스트에 대한 하나의 감성 점수 예측), 스팸 메일 분류(메일의 여러 단어에 대한 하나의 스팸/비스팸 레이블 예측) 등에 사용될 수 있습니다

In [ ]:
# 아래의 Many-to-One은 여러 개의 숫자 입력 데이터를 받고, 입력된 데이터의 총 합을 예측하는 RNN 모델을 구현합니다.
# Many-to-One
class ManyToOneRNN(nn.Module): # Many-to-One 모델 클래스 선언
    def __init__(self, input_size, hidden_size, output_size):
        super(ManyToOneRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) # RNN 모듈 생성
        self.fc = nn.Linear(hidden_size, output_size) # 선형 변환 정의

    def forward(self, x): # 순전파 함수 정의
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device) # 초기 은닉 상태를 0으로 설정
        out, _ = self.rnn(x, h0) # RNN에 입력을 전달하고 출력을 받음
        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

X = np.random.randint(1, 15, size=(50000, 6, 1)) # 입력 데이터 생성, 1~14 사이의 정수 50000개를 랜덤하게 생성
Y = np.array([np.sum(x) for x in X]) # 타겟 데이터 생성, 모든 입력 데이터의 총합이 타겟

X = torch.from_numpy(X.astype(np.float32)).to(device) # 입력 데이터를 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)).to(device) # 타겟 데이터를 텐서로 변환

model = ManyToOneRNN(input_size=1, hidden_size=50, output_size=1).to(device) # 모델 생성
criterion = nn.MSELoss() # 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘 설정

for epoch in range(4000):
    model.zero_grad()
    outputs = model(X)
    loss = criterion(outputs.squeeze(), Y)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 100 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

# Inference
X_test = torch.tensor([[[2.0], [4.0], [6.0], [8.0], [10.0], [11.0]]], dtype=torch.float32).to(device)
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: {X_test.squeeze().tolist()}")
output = round(model(X_test).item(), 1)
answer = round(sum(X_test.squeeze().tolist()), 1)

print(f"Output: {output}, 정답: {answer}")

Epoch [100/4000], Loss: 1491.8268
Epoch [200/4000], Loss: 1128.6877
Epoch [300/4000], Loss: 854.6137
Epoch [400/4000], Loss: 643.7408
Epoch [500/4000], Loss: 483.1585
Epoch [600/4000], Loss: 363.1685
Epoch [700/4000], Loss: 275.6174
Epoch [800/4000], Loss: 213.4687
Epoch [900/4000], Loss: 170.6896
Epoch [1000/4000], Loss: 142.2226
Epoch [1100/4000], Loss: 123.9612
Epoch [1200/4000], Loss: 112.6982
Epoch [1300/4000], Loss: 106.0355
Epoch [1400/4000], Loss: 102.2638
Epoch [1500/4000], Loss: 100.2245
Epoch [1600/4000], Loss: 99.1730
Epoch [1700/4000], Loss: 98.6559
Epoch [1800/4000], Loss: 98.4115
Epoch [1900/4000], Loss: 98.2956
Epoch [2000/4000], Loss: 98.2216
Epoch [2100/4000], Loss: 93.2784
Epoch [2200/4000], Loss: 58.8609
Epoch [2300/4000], Loss: 35.4755
Epoch [2400/4000], Loss: 25.5550
Epoch [2500/4000], Loss: 19.6364
Epoch [2600/4000], Loss: 15.5970
Epoch [2700/4000], Loss: 12.6368
Epoch [2800/4000], Loss: 10.4157
Epoch [2900/4000], Loss: 8.7053
Epoch [3000/4000], Loss: 7.3556
Epoc

## Many-to-Many RNN 실습
Many-to-Many RNN을 구현해보고 학습해봅시다.

Many-to-Many 구조의 모델은 여러 개의 입력에 대해 여러 개의 출력을 생성합니다.
예를 들어, 주어진 숫자 리스트의 누적 합을 예측하는 문제를 해결할 수 있습니다.
여러 개의 입력 숫자에 대해 여러 개의 출력(입력 숫자의 누적 합)을 생성합니다.
이러한 모델은 기계 번역(하나의 언어로 작성된 여러 단어에 대한 다른 언어로 작성된 여러 단어의 번역 생성), 요약 (긴 문장들을 짧은 문장으로 생성) 등에 사용될 수 있습니다

In [ ]:
#### 입력 데이터 리스트의 누적 합 리스트가 정답
# 아래의 Many-to-Many 구조는 여러 개의 숫자 입력 데이터를 받고, 입력된 데이터의 누적합을 예측하는 RNN 모델을 구현합니다.
# 예를 들어, 1,3,5를 입력으로 받으면 RNN 모델은 이들의 누적합인 1,4,9를 출력하도록 학습됩니다.

# Many-to-Many
class ManyToManyRNN(nn.Module): # ManyToManyRNN 클래스 선언
    def __init__(self, input_size, hidden_size, output_size):
        super(ManyToManyRNN, self).__init__() # nn.Module의 초기화 함수 상속
        self.hidden_size = hidden_size # 은닉 상태(hidden state)의 크기를 지정
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True) # RNN 모듈을 생성
        self.fc = nn.Linear(hidden_size, output_size) # 출력을 위한 선형 변환을 정의

    def forward(self, x): # 순전파 함수를 정의
        h0 = torch.zeros(1, x.size(0), self.hidden_size).to(x.device) # 초기 은닉 상태를 0으로 설정
        out, _ = self.rnn(x, h0) # RNN에 입력을 전달하고 출력을 받음
        out = self.fc(out) # 모든 시간 단계의 출력에 대해 선형 변환을 수행

        return out

# 데이터 생성
# 각각의 리스트는 0~30 사이의 랜덤한 정수를 가지는 5개의 정수로 구성, 이러한 리스트를 3000개 생성
X = np.array([[[np.random.randint(0, 31)] for _ in range(5)] for _ in range(3000)])
Y = np.array([np.cumsum(x) for x in X]) # 정답은 각 리스트의 누적합

X = torch.from_numpy(X.astype(np.float32)) # numpy 배열을 PyTorch 텐서로 변환
Y = torch.from_numpy(Y.astype(np.float32)) # numpy 배열을 PyTorch 텐서로 변환

model = ManyToManyRNN(1, 60, 1) # 모델 생성

criterion = nn.MSELoss() # 손실 함수 설정
optimizer = torch.optim.Adam(model.parameters()) # 최적화 알고리즘 설정

# 학습
for epoch in range(4000): # 4000번의 에폭 동안 학습
    model.zero_grad() # 기울기를 0으로 초기화
    outputs = model(X) # 모델에 입력을 전달하고 출력을 받음
    loss = criterion(outputs, Y.view_as(outputs)) # 손실 계산
    loss.backward() # 역전파 수행
    optimizer.step() # 가중치 갱신

    if (epoch+1) % 100 == 0: # 100 에폭마다 손실 출력
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 4000, loss.item()))

# 추론
X_test = torch.tensor([[[i+10] for i in range(5)]], dtype=torch.float32) # 테스트 데이터 생성
print('-' * 20, '추론 결과', '-' * 20)
print(f"Input: {list(range(10, 15))}")
output = [round(num, 1) for num in model(X_test).squeeze().tolist()] # 모델의 출력 계산
answer = list(np.cumsum(range(10, 15))) # 정답 계산

for o, a in zip(output, answer): # 모델의 출력과 정답 비교
    print(f"Output: {o}, 정답: {a}")

Epoch [100/4000], Loss: 1974.8226
Epoch [200/4000], Loss: 1566.7333
Epoch [300/4000], Loss: 1266.6932
Epoch [400/4000], Loss: 1032.9783
Epoch [500/4000], Loss: 848.1955
Epoch [600/4000], Loss: 698.7219
Epoch [700/4000], Loss: 576.6282
Epoch [800/4000], Loss: 477.1007
Epoch [900/4000], Loss: 397.0463
Epoch [1000/4000], Loss: 332.4071
Epoch [1100/4000], Loss: 279.7634
Epoch [1200/4000], Loss: 236.6477
Epoch [1300/4000], Loss: 201.1883
Epoch [1400/4000], Loss: 171.8819
Epoch [1500/4000], Loss: 147.5240
Epoch [1600/4000], Loss: 127.1678
Epoch [1700/4000], Loss: 110.0757
Epoch [1800/4000], Loss: 95.6600
Epoch [1900/4000], Loss: 83.4444
Epoch [2000/4000], Loss: 73.0572
Epoch [2100/4000], Loss: 64.1904
Epoch [2200/4000], Loss: 56.5908
Epoch [2300/4000], Loss: 50.0509
Epoch [2400/4000], Loss: 44.4053
Epoch [2500/4000], Loss: 39.4806
Epoch [2600/4000], Loss: 35.2032
Epoch [2700/4000], Loss: 31.4614
Epoch [2800/4000], Loss: 28.1821
Epoch [2900/4000], Loss: 25.2961
Epoch [3000/4000], Loss: 22.749

## 4. Gradient Vanishing / Exploding

```
💡 목차 개요 : RNN은 시퀀스가 길어질수록 처음의 정보가 끝까지 전달되지 못하는 Gradient Vanishing 문제와 Gradient가 너무 커져서 모델이 불안정해지는 Gradient Exploding 문제를 확인해보고, LSTM, GRU가 이를 완화할 수 있는지 확인해봅시다
```



RNN은 긴 시퀀스를 처리할 때 gradient vanishing 또는 exploding 문제를 겪을 수 있습니다. 이는 각 시퀀스 스텝에서 이전 스텝의 정보를 전달하는 과정에서 정보가 손실되거나, 그래디언트가 너무 커져서 파라미터가 제대로 업데이트되지 않는 문제를 의미합니다

RNN은 시퀀스가 길어질수록 처음의 정보가 끝까지 전달되지 못하는 Gradient Vanishing 문제와 Gradient가 너무 커져서 모델이 불안정해지는 Gradient Exploding 문제가 있습니다. 이를 확인하기 위해 SimpleRNN 모델에 긴 시퀀스 데이터를 학습하고, 학습률 (learning rate)을 크게 설정하여 학습을 진행해보면, RNN이 LSTM, GRU 보다 더 높은 loss를 기록하는 것을 확인할 수 있습니다. 즉, RNN에서 Gradient Exploding / Vanishing 문제를 확인할 수 있습니다

In [ ]:
# 긴 시퀀스 데이터 생성
N = 50  # number of samples
T = 20000  # sequence length
D = 10  # input dimensionality
X = np.random.randn(N, T, D)

# Targets are binary - 0 or 1
Y = np.array([1 if x.mean() > 0 else 0 for x in X]) # X의 각 샘플에 대해 평균이 0보다 크면 1, 그렇지 않으면 0

In [ ]:
# 샘플 데이터 출력
X[:2], Y[:2]

(array([[[ 0.70338203, -0.22470518, -0.25881412, ..., -0.89202369,
           1.33822481, -0.13949196],
         [ 0.04594292,  0.5721611 , -1.09314781, ...,  1.23921039,
           0.30722024,  1.14611391],
         [-0.00687995,  0.14539342, -0.2169408 , ...,  2.26734485,
          -1.18753764,  2.29719656],
         ...,
         [ 0.01304562, -0.99217194,  0.77812961, ..., -2.23289119,
          -1.5966615 , -1.10824546],
         [-1.00994753, -0.96539265,  1.01284997, ...,  0.49545009,
           0.72240584, -0.1763065 ],
         [ 0.96279892,  0.70777894,  0.82122272, ..., -1.03420976,
           0.10938281, -1.57081008]],
 
        [[ 0.5305529 , -0.50448519,  0.89366606, ...,  1.11510732,
           1.49382758, -1.37135707],
         [ 0.29154474, -0.02630598,  0.50583689, ...,  0.54472606,
           0.63576751, -0.62527457],
         [-0.48608331,  0.26244451, -0.38990753, ..., -1.89786405,
          -0.75340474, -0.30848156],
         ...,
         [-1.23670985,  0.5072984

In [ ]:
class SimpleRNN(nn.Module): # SimpleRNN 클래스 선언
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(SimpleRNN, self).__init__() # nn.Module의 초기화 함수 상속
        self.M = n_hidden # 은닉 상태(hidden state)의 크기를 지정
        self.D = n_inputs # 입력 차원의 크기 지정
        self.K = n_outputs # 출력 차원의 크기 지정
        self.rnn = nn.RNN( # RNN 모듈을 생성
            input_size=self.D, # 입력 차원의 크기 지정
            hidden_size=self.M, # 은닉 상태의 크기 지정
            nonlinearity='tanh', # 활성화 함수로 tanh를 사용
            batch_first=True) # 배치 차원이 먼저 오도록 설정
        self.fc = nn.Linear(self.M, self.K) # 출력을 위한 선형 변환을 정의

    def forward(self, X): # 순전파 함수를 정의
        # initial hidden states
        h0 = torch.zeros(1, X.size(0), self.M).to(X.device) # 초기 은닉 상태를 0으로 설정

        # get RNN unit output
        out, _ = self.rnn(X, h0) # RNN에 입력을 전달하고 출력을 받음

        # we only want h(T) at the final time step
        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

# RNN 학습
M = 5  # hidden layer size
K = 1  # output dimensionality

model = SimpleRNN(n_inputs=D, n_hidden=M, n_outputs=K).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

inputs = torch.from_numpy(X.astype(np.float32)).to(device)
targets = torch.from_numpy(Y.astype(np.float32)).to(device)

for epoch in range(120):
    model.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs.squeeze(), targets)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 120, loss.item()))

Epoch [10/120], Loss: 0.6784
Epoch [20/120], Loss: 0.6453
Epoch [30/120], Loss: 0.6098
Epoch [40/120], Loss: 0.5712
Epoch [50/120], Loss: 0.5265
Epoch [60/120], Loss: 0.4760
Epoch [70/120], Loss: 0.4189
Epoch [80/120], Loss: 0.3586
Epoch [90/120], Loss: 0.3110
Epoch [100/120], Loss: 0.2744
Epoch [110/120], Loss: 0.2433
Epoch [120/120], Loss: 0.2126


In [ ]:
class LSTM(nn.Module): # LSTM 클래스 선언
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(LSTM, self).__init__() # nn.Module의 초기화 함수 상속
        self.D = n_inputs # 입력 차원의 크기 지정
        self.M = n_hidden # 은닉 상태(hidden state)의 크기를 지정
        self.K = n_outputs # 출력 차원의 크기 지정
        self.lstm = nn.LSTM( # LSTM 모듈을 생성
            input_size=self.D, # 입력 차원의 크기 지정
            hidden_size=self.M, # 은닉 상태의 크기 지정
            batch_first=True) # 배치 차원이 먼저 오도록 설정
        self.fc = nn.Linear(self.M, self.K) # 출력을 위한 선형 변환을 정의

    def forward(self, X): # 순전파 함수를 정의
        # initial hidden states
        h0 = torch.zeros(1, X.size(0), self.M).to(X.device) # 초기 은닉 상태를 0으로 설정
        c0 = torch.zeros(1, X.size(0), self.M).to(X.device) # LSTM의 초기 cell state를 0으로 설정

        # get RNN unit output
        out, _ = self.lstm(X, (h0, c0)) # LSTM에 입력을 전달하고 출력을 받음

        # we only want h(T) at the final time step
        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

# LSTM 학습
model = LSTM(n_inputs=D, n_hidden=M, n_outputs=K).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

for epoch in range(120):
    model.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs.squeeze(), targets)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 120, loss.item()))

Epoch [10/120], Loss: 0.6782
Epoch [20/120], Loss: 0.6627
Epoch [30/120], Loss: 0.6376
Epoch [40/120], Loss: 0.5956
Epoch [50/120], Loss: 0.5315
Epoch [60/120], Loss: 0.4498
Epoch [70/120], Loss: 0.3682
Epoch [80/120], Loss: 0.2911
Epoch [90/120], Loss: 0.2243
Epoch [100/120], Loss: 0.1701
Epoch [110/120], Loss: 0.1336
Epoch [120/120], Loss: 0.1058


In [ ]:
class GRU(nn.Module): # GRU 클래스 선언
    def __init__(self, n_inputs, n_hidden, n_outputs):
        super(GRU, self).__init__() # nn.Module의 초기화 함수 상속
        self.D = n_inputs # 입력 차원의 크기 지정
        self.M = n_hidden # 은닉 상태(hidden state)의 크기를 지정
        self.K = n_outputs # 출력 차원의 크기 지정
        self.gru = nn.GRU( # GRU 모듈을 생성
            input_size=self.D, # 입력 차원의 크기 지정
            hidden_size=self.M, # 은닉 상태의 크기 지정
            batch_first=True) # 배치 차원이 먼저 오도록 설정
        self.fc = nn.Linear(self.M, self.K) # 출력을 위한 선형 변환을 정의

    def forward(self, X): # 순전파 함수를 정의
        # initial hidden states
        h0 = torch.zeros(1, X.size(0), self.M).to(X.device) # 초기 은닉 상태를 0으로 설정

        # get RNN unit output
        out, _ = self.gru(X, h0) # GRU에 입력을 전달하고 출력을 받음

        # we only want h(T) at the final time step
        out = self.fc(out[:, -1, :]) # 마지막 시간 단계의 출력만 사용하여 선형 변환을 수행

        return out

# GRU 학습
model = GRU(n_inputs=D, n_hidden=M, n_outputs=K).to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=5e-3)

for epoch in range(120):
    model.zero_grad()
    outputs = model(inputs)
    loss = criterion(outputs.squeeze(), targets)
    loss.backward()
    optimizer.step()

    if (epoch+1) % 10 == 0:
        print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, 120, loss.item()))

Epoch [10/120], Loss: 0.6681
Epoch [20/120], Loss: 0.6286
Epoch [30/120], Loss: 0.5801
Epoch [40/120], Loss: 0.5174
Epoch [50/120], Loss: 0.4441
Epoch [60/120], Loss: 0.3668
Epoch [70/120], Loss: 0.2941
Epoch [80/120], Loss: 0.2354
Epoch [90/120], Loss: 0.1905
Epoch [100/120], Loss: 0.1530
Epoch [110/120], Loss: 0.1218
Epoch [120/120], Loss: 0.0974


In [ ]:
print(torch.__version__)
print(np.__version__)


# Reference

- [stanford.edu](https://stanford.edu/~shervine/teaching/cs-230/cheatsheet-recurrent-neural-networks)

## Required Package

+ torch == 2.0.1+cu118
+ numpy == 1.22.4


